In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf

In [ ]:
from PIL import Image
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = np.array(img)
        features.append(img)

    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)
    return features

In [ ]:
import os
def load_and_preprocess_data(base_dir):
    image_paths = []
    age_labels = []
    gender_labels = []

    for filename in tqdm(os.listdir(base_dir)):
        image_path = os.path.join(base_dir, filename)
        temp = filename.split('_')
        age = int(temp[0])
        gender = int(temp[1])
        image_paths.append(image_path)
        age_labels.append(age)
        gender_labels.append(gender)

    df = pd.DataFrame()
    df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels

    X = extract_features(df['image'])
    X = X / 255.0
    y_gender = np.array(df['gender'])
    y_age = np.array(df['age'])

    return X, y_gender, y_age

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input
def build_model(input_shape):
    inputs = Input(input_shape)

    conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
    maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
    conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxp_1)
    maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
    conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(maxp_2)
    maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)
    conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu')(maxp_3)
    maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)

    flatten = Flatten()(maxp_4)

    dense_1 = Dense(256, activation='relu')(flatten)
    dense_2 = Dense(256, activation='relu')(flatten)

    dropout_1 = Dropout(0.3)(dense_1)
    dropout_2 = Dropout(0.3)(dense_2)

    output_1 = Dense(1, activation='sigmoid', name='gender_out')(dropout_1)
    output_2 = Dense(1, activation='relu', name='age_out')(dropout_2)

    model = Model(inputs=[inputs], outputs=[output_1, output_2])

    model.compile(loss=['binary_crossentropy', 'mae'], optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
def train_model(model, X, y_gender, y_age):
    history = model.fit(x=X, y=[y_gender, y_age], batch_size=32, epochs=30, validation_split=0.2)
    return history

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 7))

    # Plot gender training accuracy vs. validation accuracy
    plt.subplot(2, 2, 1)
    plt.plot(history.history['gender_out_accuracy'], label='Gender Training Accuracy')
    plt.plot(history.history['val_gender_out_accuracy'], label='Gender Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Gender Training Accuracy vs. Validation Accuracy')

    # Plot gender training loss vs. validation loss
    plt.subplot(2, 2, 2)
    plt.plot(history.history['gender_out_loss'], label='Gender Training Loss')
    plt.plot(history.history['val_gender_out_loss'], label='Gender Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Gender Training Loss vs. Validation Loss')

    # Plot age training loss vs. validation loss
    plt.subplot(2, 2, 3)
    plt.plot(history.history['age_out_loss'], label='Age Training Loss')
    plt.plot(history.history['val_age_out_loss'], label='Age Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Age Training Loss vs. Validation Loss')

    plt.tight_layout()
    plt.show()


In [ ]:
if __name__ == "__main__":
    BASE_DIR = "/kaggle/input/utkface-new/UTKFace"
    
    X, y_gender, y_age = load_and_preprocess_data(BASE_DIR)
    
    input_shape = (128, 128, 1)
    model = build_model(input_shape)
    
    history = train_model(model, X, y_gender, y_age)
    
    plot_training_history(history)

In [ ]:
def predict_and_visualize_results(model, X, y_gender, y_age, image_indices):
    gender_dict = {0: 'male', 1: 'female'}
    results = []

    for image_index in image_indices:
        original_gender = gender_dict[y_gender[image_index]]
        original_age = y_age[image_index]

        # Predict gender and age for the given image_index
        predictions = model.predict(X[image_index].reshape(1, 128, 128, 1))
        predicted_gender = gender_dict[round(predictions[0][0][0])]
        predicted_age = round(predictions[1][0][0])

        results.append((original_gender, original_age, predicted_gender, predicted_age))

    return results

image_indices = [200, 1300, 13200, 19400]
results = predict_and_visualize_results(model, X, y_gender, y_age, image_indices)

# Display all results
for i, (original_gender, original_age, predicted_gender, predicted_age) in enumerate(results):
    plt.figure(figsize=(4, 4))
    plt.imshow(X[image_indices[i]].reshape(128, 128), cmap='gray')
    plt.axis('off')
    plt.title(f"Original Gender: {original_gender}, Original Age: {original_age}\nPredicted Gender: {predicted_gender}, Predicted Age: {predicted_age}")
    plt.show()
